In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

## Get dimensions of each image

In [2]:
df = pd.DataFrame()

with open('annotations/person_keypoints_train2017.json') as json_data:
    data = json.load(json_data)
    
file_names = []
heights = []
widths = []
for i in data['images']:
    file_names.append(i['file_name'])
    heights.append(i['height'])
    widths.append(i['width'])
    
with open('annotations/person_keypoints_val2017.json') as json_data:
    data = json.load(json_data)
    
for i in data['images']:
    file_names.append(i['file_name'])
    heights.append(i['height'])
    widths.append(i['width'])
    
df['file_name'] = file_names
df['height'] = heights
df['width'] = widths
df

,file_name,height,width
0,000000391895.jpg,360,640
1,000000522418.jpg,480,640
2,000000184613.jpg,336,500
3,000000318219.jpg,640,556
4,000000554625.jpg,640,426
...,...,...,...
123282,000000512403.jpg,640,529
123283,000000168974.jpg,500,375
123284,000000552775.jpg,500,375
123285,000000394940.jpg,640,426


In [3]:
df.to_csv('img_info.csv',index=False)

## COCO bbox (train)

In [4]:
with open('annotations/person_keypoints_train2017.json') as json_data:
    data = json.load(json_data)

tempDf = pd.DataFrame(columns=['image_id','bbox'])
image_ids = []
for i in data['annotations']:
    image_ids.append(i['image_id'])
    
tempDf['image_id'] = image_ids

bboxes = []
for i in data['annotations']:
    bboxes.append(i['bbox'])
    
tempDf['bbox'] = bboxes
tempDf

,image_id,bbox
0,537548,"[267.03, 104.32, 229.19, 320]"
1,117891,"[206.77, 1.44, 433.23, 408.73]"
2,120021,"[276.12, 0.29, 61.18, 118.73]"
3,403255,"[355.87, 118.43, 30.84, 42.87]"
4,209468,"[178.38, 120.54, 114.59, 354.6]"
...,...,...
262460,128732,"[9, 284, 560, 141]"
262461,489186,"[298, 0, 341, 127]"
262462,390883,"[40, 104, 394, 43]"
262463,554743,"[275, 207, 153, 148]"


In [5]:
tempDf.to_csv('coco_bbox.csv', index=False)

## Add training labels

In [6]:
df = pd.read_csv('coco_bbox.csv')
df['image_file'] = df['image_id']
df['label_file'] = df['image_id']
df

,image_id,bbox,image_file,label_file
0,537548,"[267.03, 104.32, 229.19, 320]",537548,537548
1,117891,"[206.77, 1.44, 433.23, 408.73]",117891,117891
2,120021,"[276.12, 0.29, 61.18, 118.73]",120021,120021
3,403255,"[355.87, 118.43, 30.84, 42.87]",403255,403255
4,209468,"[178.38, 120.54, 114.59, 354.6]",209468,209468
...,...,...,...,...
262460,128732,"[9, 284, 560, 141]",128732,128732
262461,489186,"[298, 0, 341, 127]",489186,489186
262462,390883,"[40, 104, 394, 43]",390883,390883
262463,554743,"[275, 207, 153, 148]",554743,554743


In [7]:
df = df.astype({'image_file':'string'})
df = df.astype({'label_file':'string'})
df.dtypes

image_id       int64
bbox          object
image_file    string
label_file    string
dtype: object

In [8]:
image_files = df['image_file']
label_files = df['label_file']

for i in range(len(image_files)):
    s = image_files[i]
    n = len(s)
    zeros = '0' * (12-n)
    new_s = zeros + s + '.jpg'
    image_files[i] = new_s
    
for i in range(len(label_files)):
    s = label_files[i]
    n = len(s)
    zeros = '0' * (12-n)
    new_s = zeros + s + '.txt'
    label_files[i] = new_s
    
df['image_file'] = image_files
df['label_file'] = label_files
df

,image_id,bbox,image_file,label_file
0,537548,"[267.03, 104.32, 229.19, 320]",000000537548.jpg,000000537548.txt
1,117891,"[206.77, 1.44, 433.23, 408.73]",000000117891.jpg,000000117891.txt
2,120021,"[276.12, 0.29, 61.18, 118.73]",000000120021.jpg,000000120021.txt
3,403255,"[355.87, 118.43, 30.84, 42.87]",000000403255.jpg,000000403255.txt
4,209468,"[178.38, 120.54, 114.59, 354.6]",000000209468.jpg,000000209468.txt
...,...,...,...,...
262460,128732,"[9, 284, 560, 141]",000000128732.jpg,000000128732.txt
262461,489186,"[298, 0, 341, 127]",000000489186.jpg,000000489186.txt
262462,390883,"[40, 104, 394, 43]",000000390883.jpg,000000390883.txt
262463,554743,"[275, 207, 153, 148]",000000554743.jpg,000000554743.txt


In [9]:
df['to_put'] = df['bbox']
df = df.astype({'to_put':'string'})
df.dtypes

image_id       int64
bbox          object
image_file    string
label_file    string
to_put        string
dtype: object

In [10]:
img_info = pd.read_csv('img_info.csv', index_col='file_name')

boxes = df['to_put']
for i in range(len(boxes)):
    x = eval(boxes[i])
    img_width = img_info.loc[image_files[i]]['width']
    img_height = img_info.loc[image_files[i]]['height']
    box_width = x[2]
    box_height = x[3]
    box_startX = x[0]
    box_startY = x[1]
    box_midX = (box_startX + (box_startX + box_width)) / 2
    box_midY = (box_startY + (box_startY + box_height)) / 2
    
    box_width /= img_width
    box_midX /= img_width
    box_height /= img_height
    box_midY /= img_height
    
    x = [box_midX, box_midY, box_width, box_height]
    for j in range(len(x)):
        if x[j]<=0:
            x[j] = 0.000001
        if x[j]>1:
            x[j] = 1
    for j in x:
        if j<=0 or j>1:
            print(j)
    y = [str(z) for z in x]
    s = ' '.join(y)
    s = '0 ' + s
    boxes[i] = s

boxes

0         0 0.5962890625 0.5506666666666666 0.358109375 ...
1         0 0.6615390624999999 0.4819789227166277 0.6769...
2         0 0.47923437500000005 0.12428125000000002 0.09...
3         0 0.580140625 0.36423177083333336 0.0481875 0....
4          0 0.36824218750000004 0.6205 0.179046875 0.73875
                                ...                        
262460    0 0.4515625 0.8321596244131455 0.875 0.3309859...
262461    0 0.73203125 0.1697860962566845 0.5328125 0.33...
262462    0 0.474 0.3826219512195122 0.788 0.13109756097...
262463    0 0.54921875 0.7827298050139275 0.2390625 0.41...
262464    0 0.66484375 0.7049180327868853 0.6203125 0.19...
Name: to_put, Length: 262465, dtype: string

In [11]:
df['to_put'] = boxes
df

,image_id,bbox,image_file,label_file,to_put
0,537548,"[267.03, 104.32, 229.19, 320]",000000537548.jpg,000000537548.txt,0 0.5962890625 0.5506666666666666 0.358109375 ...
1,117891,"[206.77, 1.44, 433.23, 408.73]",000000117891.jpg,000000117891.txt,0 0.6615390624999999 0.4819789227166277 0.6769...
2,120021,"[276.12, 0.29, 61.18, 118.73]",000000120021.jpg,000000120021.txt,0 0.47923437500000005 0.12428125000000002 0.09...
3,403255,"[355.87, 118.43, 30.84, 42.87]",000000403255.jpg,000000403255.txt,0 0.580140625 0.36423177083333336 0.0481875 0....
4,209468,"[178.38, 120.54, 114.59, 354.6]",000000209468.jpg,000000209468.txt,0 0.36824218750000004 0.6205 0.179046875 0.73875
...,...,...,...,...,...
262460,128732,"[9, 284, 560, 141]",000000128732.jpg,000000128732.txt,0 0.4515625 0.8321596244131455 0.875 0.3309859...
262461,489186,"[298, 0, 341, 127]",000000489186.jpg,000000489186.txt,0 0.73203125 0.1697860962566845 0.5328125 0.33...
262462,390883,"[40, 104, 394, 43]",000000390883.jpg,000000390883.txt,0 0.474 0.3826219512195122 0.788 0.13109756097...
262463,554743,"[275, 207, 153, 148]",000000554743.jpg,000000554743.txt,0 0.54921875 0.7827298050139275 0.2390625 0.41...


In [12]:
for index, row in df.iterrows():
    f = open('labels/' + row['label_file'],'a+')
    f.write(row['to_put']+'\n')
    f.close()

## Creating train.csv

In [13]:
full_df = pd.DataFrame()

img_files = list(set(df['image_file']))

full_df['image_file'] = img_files
full_df['label_file'] = full_df['image_file']

label_files = full_df['label_file']

for i in range(len(label_files)):
    s = label_files[i]
    new_s = s[:-3] + 'txt'
    label_files[i] = new_s

full_df['label_file'] = label_files
full_df

,image_file,label_file
0,000000438946.jpg,000000438946.txt
1,000000346642.jpg,000000346642.txt
2,000000360045.jpg,000000360045.txt
3,000000367429.jpg,000000367429.txt
4,000000292183.jpg,000000292183.txt
...,...,...
64110,000000301769.jpg,000000301769.txt
64111,000000091188.jpg,000000091188.txt
64112,000000158913.jpg,000000158913.txt
64113,000000111661.jpg,000000111661.txt


In [14]:
full_df.to_csv('train.csv', index=False)

## COCO bbox (test)

In [15]:
with open('annotations/person_keypoints_val2017.json') as json_data:
    data = json.load(json_data)

tempDf = pd.DataFrame(columns=['image_id','bbox'])
image_ids = []
for i in data['annotations']:
    image_ids.append(i['image_id'])
    
tempDf['image_id'] = image_ids

bboxes = []
for i in data['annotations']:
    bboxes.append(i['bbox'])
    
tempDf['bbox'] = bboxes
tempDf

,image_id,bbox
0,425226,"[73.35, 206.02, 300.58, 372.5]"
1,440475,"[491.91, 183.51, 34.85, 73.4]"
2,266400,"[36.32, 0.59, 38.56, 53.86]"
3,69213,"[276.35, 9.6, 244.69, 400.13]"
4,555050,"[97.1, 185.73, 28.66, 27.72]"
...,...,...
10999,259690,"[27, 98, 225, 114]"
11000,145020,"[27, 201, 209, 68]"
11001,562818,"[353, 229, 180, 39]"
11002,439994,"[0, 0, 427, 458]"


In [16]:
tempDf.to_csv('coco_bbox_test.csv', index=False)

## Add test labels

In [17]:
df = pd.read_csv('coco_bbox_test.csv')
df['image_file'] = df['image_id']
df['label_file'] = df['image_id']
df

,image_id,bbox,image_file,label_file
0,425226,"[73.35, 206.02, 300.58, 372.5]",425226,425226
1,440475,"[491.91, 183.51, 34.85, 73.4]",440475,440475
2,266400,"[36.32, 0.59, 38.56, 53.86]",266400,266400
3,69213,"[276.35, 9.6, 244.69, 400.13]",69213,69213
4,555050,"[97.1, 185.73, 28.66, 27.72]",555050,555050
...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",259690,259690
11000,145020,"[27, 201, 209, 68]",145020,145020
11001,562818,"[353, 229, 180, 39]",562818,562818
11002,439994,"[0, 0, 427, 458]",439994,439994


In [18]:
df = df.astype({'image_file':'string'})
df = df.astype({'label_file':'string'})
df.dtypes

image_id       int64
bbox          object
image_file    string
label_file    string
dtype: object

In [19]:
image_files = df['image_file']
label_files = df['label_file']

for i in range(len(image_files)):
    s = image_files[i]
    n = len(s)
    zeros = '0' * (12-n)
    new_s = zeros + s + '.jpg'
    image_files[i] = new_s
    
for i in range(len(label_files)):
    s = label_files[i]
    n = len(s)
    zeros = '0' * (12-n)
    new_s = zeros + s + '.txt'
    label_files[i] = new_s
    
df['image_file'] = image_files
df['label_file'] = label_files
df

,image_id,bbox,image_file,label_file
0,425226,"[73.35, 206.02, 300.58, 372.5]",000000425226.jpg,000000425226.txt
1,440475,"[491.91, 183.51, 34.85, 73.4]",000000440475.jpg,000000440475.txt
2,266400,"[36.32, 0.59, 38.56, 53.86]",000000266400.jpg,000000266400.txt
3,69213,"[276.35, 9.6, 244.69, 400.13]",000000069213.jpg,000000069213.txt
4,555050,"[97.1, 185.73, 28.66, 27.72]",000000555050.jpg,000000555050.txt
...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",000000259690.jpg,000000259690.txt
11000,145020,"[27, 201, 209, 68]",000000145020.jpg,000000145020.txt
11001,562818,"[353, 229, 180, 39]",000000562818.jpg,000000562818.txt
11002,439994,"[0, 0, 427, 458]",000000439994.jpg,000000439994.txt


In [20]:
df['to_put'] = df['bbox']
df = df.astype({'to_put':'string'})
df.dtypes

image_id       int64
bbox          object
image_file    string
label_file    string
to_put        string
dtype: object

In [21]:
img_info = pd.read_csv('img_info.csv', index_col='file_name')

boxes = df['to_put']
for i in range(len(boxes)):
    x = eval(boxes[i])
    img_width = img_info.loc[image_files[i]]['width']
    img_height = img_info.loc[image_files[i]]['height']
    box_width = x[2]
    box_height = x[3]
    box_startX = x[0]
    box_startY = x[1]
    box_midX = (box_startX + (box_startX + box_width)) / 2
    box_midY = (box_startY + (box_startY + box_height)) / 2
    
    box_width /= img_width
    box_midX /= img_width
    box_height /= img_height
    box_midY /= img_height
    
    x = [box_midX, box_midY, box_width, box_height]
    for j in range(len(x)):
        if x[j]<=0:
            x[j] = 0.000001
        if x[j]>1:
            x[j] = 1
    for j in x:
        if j<=0 or j>1:
            print(j)
    y = [str(z) for z in x]
    s = ' '.join(y)
    s = '0 ' + s
    boxes[i] = s

boxes

0        0 0.46591666666666665 0.612921875 0.6262083333...
1        0 0.7958359375 0.5157142857142857 0.0544531250...
2        0 0.086875 0.0694949494949495 0.06025000000000...
3        0 0.6229609375 0.4910187353629977 0.382328125 ...
4        0 0.22285999999999997 0.6198447204968943 0.057...
                               ...                        
10999     0 0.21796875 0.3229166666666667 0.3515625 0.2375
11000    0 0.20546875 0.6509695290858726 0.3265625 0.18...
11001       0 0.6921875 0.5177083333333333 0.28125 0.08125
11002    0 0.49883177570093457 0.3578125 0.997663551401...
11003    0 0.49921875 0.5352112676056338 0.9984375 0.91...
Name: to_put, Length: 11004, dtype: string

In [22]:
df['to_put'] = boxes
df

,image_id,bbox,image_file,label_file,to_put
0,425226,"[73.35, 206.02, 300.58, 372.5]",000000425226.jpg,000000425226.txt,0 0.46591666666666665 0.612921875 0.6262083333...
1,440475,"[491.91, 183.51, 34.85, 73.4]",000000440475.jpg,000000440475.txt,0 0.7958359375 0.5157142857142857 0.0544531250...
2,266400,"[36.32, 0.59, 38.56, 53.86]",000000266400.jpg,000000266400.txt,0 0.086875 0.0694949494949495 0.06025000000000...
3,69213,"[276.35, 9.6, 244.69, 400.13]",000000069213.jpg,000000069213.txt,0 0.6229609375 0.4910187353629977 0.382328125 ...
4,555050,"[97.1, 185.73, 28.66, 27.72]",000000555050.jpg,000000555050.txt,0 0.22285999999999997 0.6198447204968943 0.057...
...,...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",000000259690.jpg,000000259690.txt,0 0.21796875 0.3229166666666667 0.3515625 0.2375
11000,145020,"[27, 201, 209, 68]",000000145020.jpg,000000145020.txt,0 0.20546875 0.6509695290858726 0.3265625 0.18...
11001,562818,"[353, 229, 180, 39]",000000562818.jpg,000000562818.txt,0 0.6921875 0.5177083333333333 0.28125 0.08125
11002,439994,"[0, 0, 427, 458]",000000439994.jpg,000000439994.txt,0 0.49883177570093457 0.3578125 0.997663551401...


In [23]:
for index, row in df.iterrows():
    f = open('labels/' + row['label_file'],'a+')
    f.write(row['to_put']+'\n')
    f.close()

## Creating test.csv

In [24]:
full_df = pd.DataFrame()

img_files = list(set(df['image_file']))

full_df['image_file'] = img_files
full_df['label_file'] = full_df['image_file']

label_files = full_df['label_file']

for i in range(len(label_files)):
    s = label_files[i]
    new_s = s[:-3] + 'txt'
    label_files[i] = new_s

full_df['label_file'] = label_files
full_df

,image_file,label_file
0,000000326128.jpg,000000326128.txt
1,000000019432.jpg,000000019432.txt
2,000000551820.jpg,000000551820.txt
3,000000530854.jpg,000000530854.txt
4,000000500270.jpg,000000500270.txt
...,...,...
2688,000000313588.jpg,000000313588.txt
2689,000000246436.jpg,000000246436.txt
2690,000000374083.jpg,000000374083.txt
2691,000000161032.jpg,000000161032.txt


In [25]:
full_df.to_csv('test.csv', index=False)

Next step: Manually move the images to 'images' folder